# **📁 向量資料庫壓縮檔說明**
本階段分為兩部分：

1. **準備資料**：包含手動下載或透過網址（URL）自動下載原始檔案。
2. **建立壓縮檔**：將處理後的向量資料建立成 FAISS 向量資料庫，並壓縮儲存，供後續檢索系統載入使用。

此壓縮檔可直接用於載入語意索引，加速回答流程。

---


### **1. 準備資料**

#### **1.1 MedlinePlus 健康資料下載與解析程式**

這段程式碼負責從 MedlinePlus 官方網站自動下載多份健康相關 XML 檔案，並解析出主題摘要內容，為後續向量化與檢索系統做準備。

#### 🔧 功能說明

1. **資料夾建立**  
   自動建立 `medlineplus_data/` 資料夾，用於儲存下載與解析後的檔案。

2. **XML 檔案下載**  
   透過 `requests` 套件下載 MedlinePlus 官方發布之多份 XML，包括：
   - 健康主題（HealthTopics）
   - 主題分類（TopicGroup）
   - 營養/運動定義（如 Vitamins、Minerals、Nutrition、Fitness）

3. **主題摘要解析**  
   僅針對 HealthTopics 檔案進行解析，擷取每筆健康主題的：
   - 標題（Title）
   - 完整摘要（FullSummary）
   - 網頁連結（URL）

4. **JSON 輸出**  
   將解析後的健康主題資料儲存為 `HealthTopics_parsed.json`，供向量化模型使用。

#### ✅ 執行結果

- 所有 XML 檔案會儲存在 `medlineplus_data/`
- 成功解析之主題摘要將輸出為可直接使用的 JSON 格式


In [ ]:
import os
import requests
import xml.etree.ElementTree as ET
import json

# ========== 基本參數設定 ==========
target_dir = "medlineplus_data"
os.makedirs(target_dir, exist_ok=True)

# 所需 XML 檔案（可自行擴增）
files_to_download = {
    "HealthTopics": "https://medlineplus.gov/xml/mplus_topics_2025-05-27.xml",
    "TopicGroup": "https://medlineplus.gov/xml/mplus_topic_groups_2025-05-27.xml",
    "Fitness": "https://medlineplus.gov/xml/fitnessdefinitions.xml",
    "GeneralHealth": "https://medlineplus.gov/xml/generalhealthdefinitions.xml",
    "Minerals": "https://medlineplus.gov/xml/mineralsdefinitions.xml",
    "Nutrition": "https://medlineplus.gov/xml/nutritiondefinitions.xml",
    "Vitamins": "https://medlineplus.gov/xml/vitaminsdefinitions.xml"
}


# ========== 步驟一：下載 XML ==========
def download_file(url):
    filename = url.split("/")[-1]
    local_path = os.path.join(target_dir, filename)
    if not os.path.exists(local_path):
        print(f"⬇️ 下載中: {url}")
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(url, headers=headers, stream=True)
        if response.status_code == 200:
            with open(local_path, "wb") as f:
                f.write(response.content)
            print(f"✅ 下載完成: {filename}")
        else:
            print(f"❌ 無法下載: {url}（HTTP {response.status_code}）")
    else:
        print(f"✅ 已存在: {filename}")
    return local_path

# ========== 步驟二：解析主題檔 XML ==========
def parse_health_topics(xml_path):
    print(f"📖 解析檔案: {xml_path}")
    tree = ET.parse(xml_path)
    root = tree.getroot()

    namespace = {"ns": "https://medlineplus.gov/xml/healthtopics"}
    data = []

    for topic in root.findall("ns:HealthTopic", namespace):
        title = topic.findtext("ns:Title", default="", namespaces=namespace)
        summary = topic.findtext("ns:FullSummary", default="", namespaces=namespace)
        url = topic.findtext("ns:URL", default="", namespaces=namespace)
        if title and summary:
            data.append({
                "title": title.strip(),
                "summary": summary.strip(),
                "url": url.strip()
            })

    print(f"✅ 完成解析，共 {len(data)} 筆")
    return data

# ========== 主執行程序 ==========
if __name__ == "__main__":
    parsed_results = {}

    for key, url in files_to_download.items():
        file_path = download_file(url)

        if key == "HealthTopics":
            topics = parse_health_topics(file_path)
            parsed_results[key] = topics

            # 儲存為 JSON，供後續向量化使用
            json_path = os.path.join(target_dir, f"{key}_parsed.json")
            with open(json_path, 'w', encoding='utf-8') as f:
                json.dump(topics, f, ensure_ascii=False, indent=2)
            print(f"💾 已儲存至: {json_path}")

    print("\n✅ 所有檔案已處理完成，待後續向量化使用。")


⬇️ 下載中: https://medlineplus.gov/xml/mplus_topics_2025-05-27.xml
✅ 下載完成: mplus_topics_2025-05-27.xml
📖 解析檔案: medlineplus_data/mplus_topics_2025-05-27.xml
✅ 完成解析，共 0 筆
💾 已儲存至: medlineplus_data/HealthTopics_parsed.json
⬇️ 下載中: https://medlineplus.gov/xml/mplus_topic_groups_2025-05-27.xml
✅ 下載完成: mplus_topic_groups_2025-05-27.xml
⬇️ 下載中: https://medlineplus.gov/xml/fitnessdefinitions.xml
✅ 下載完成: fitnessdefinitions.xml
⬇️ 下載中: https://medlineplus.gov/xml/generalhealthdefinitions.xml
✅ 下載完成: generalhealthdefinitions.xml
⬇️ 下載中: https://medlineplus.gov/xml/mineralsdefinitions.xml
✅ 下載完成: mineralsdefinitions.xml
⬇️ 下載中: https://medlineplus.gov/xml/nutritiondefinitions.xml
✅ 下載完成: nutritiondefinitions.xml
⬇️ 下載中: https://medlineplus.gov/xml/vitaminsdefinitions.xml
✅ 下載完成: vitaminsdefinitions.xml

✅ 所有檔案已處理完成，待後續向量化使用。


#### **1.2 健康資料 JSON/XML 轉純文字檔**

此腳本將前一階段取得的 MedlinePlus 健康資料（JSON 或 XML 檔案）轉換為 `.txt` 格式，方便後續用於向量化處理。

#### 🔧 功能簡述

1. **處理 HealthTopics JSON**
   - 擷取每筆主題的標題、摘要與來源網址
   - 輸出為個別純文字檔（每篇一檔）

2. **處理其他 XML 定義檔**
   - 擷取所有文字節點內容
   - 整合為單一文字檔，檔名與原 XML 對應

3. **輸出資料夾**
   - 所有 `.txt` 檔會儲存於 `parsed_txt/` 資料夾中


In [ ]:
import os
import json
import xml.etree.ElementTree as ET

# ========== 參數設定 ==========
input_folder = "medlineplus_data"
output_folder = "parsed_txt"
os.makedirs(output_folder, exist_ok=True)

# ========== 處理 JSON 檔案（如 HealthTopics） ==========
def json_to_txt(json_path, txt_output_dir):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    count = 0
    for item in data:
        title = item.get("title", "").strip()
        summary = item.get("summary", "").strip()
        url = item.get("url", "").strip()

        if title and summary:
            filename = f"{title[:50].replace('/', '_')}.txt"
            filepath = os.path.join(txt_output_dir, filename)
            with open(filepath, 'w', encoding='utf-8') as out:
                out.write(f"{title}\n\n{summary}\n\n來源: {url}")
            count += 1

    print(f"✅ 已轉換 JSON 檔 {os.path.basename(json_path)}，共 {count} 篇文章")

# ========== 處理一般 XML 檔案（如 vitaminsdefinitions.xml 等） ==========
def xml_to_txt(xml_path, txt_output_dir):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    filename = os.path.basename(xml_path).replace(".xml", ".txt")
    output_path = os.path.join(txt_output_dir, filename)

    with open(output_path, 'w', encoding='utf-8') as f:
        for elem in root.iter():
            if elem.text and elem.text.strip():
                f.write(elem.text.strip() + "\n\n")
    print(f"✅ 已轉換 XML 檔 {os.path.basename(xml_path)} 為純文字")

# ========== 主轉換流程 ==========
if __name__ == "__main__":
    for file in os.listdir(input_folder):
        full_path = os.path.join(input_folder, file)

        if file.endswith(".json"):
            json_to_txt(full_path, output_folder)

        elif file.endswith(".xml"):
            # 避免重複轉換已解析過的 HealthTopics
            if file != "mplus_topics_2025-05-27.xml":
                xml_to_txt(full_path, output_folder)

    print("\n📁 所有 json/xml 皆已轉為 txt 檔，儲存於:", output_folder)


✅ 已轉換 JSON 檔 HealthTopics_parsed.json，共 0 篇文章
✅ 已轉換 XML 檔 mplus_topic_groups_2025-05-27.xml 為純文字
✅ 已轉換 XML 檔 mineralsdefinitions.xml 為純文字
✅ 已轉換 XML 檔 nutritiondefinitions.xml 為純文字
✅ 已轉換 XML 檔 generalhealthdefinitions.xml 為純文字
✅ 已轉換 XML 檔 vitaminsdefinitions.xml 為純文字
✅ 已轉換 XML 檔 fitnessdefinitions.xml 為純文字

📁 所有 json/xml 皆已轉為 txt 檔，儲存於: parsed_txt


#### **1.3 資料來源補充說明**

除了自動擷取的 MedlinePlus 資料外，本專案亦**手動下載並整理了來自 Mayo Clinic 及台灣衛福部食藥署的相關健康資料**，以提升內容多元性與本地適用性。這些補充資料已納入向量資料庫中，成為系統回應時的重要知識依據之一。


---

### **2. 準備建立向量資料庫**

#### **2.1 上傳檔案資料夾建立與套件安裝**

此區塊用於準備檔案上傳環境與安裝向量資料庫建構所需套件。


#### 📁 建立上傳資料夾
- 建立名為 `uploaded_docs` 的資料夾，用來存放上傳的 PDF、Word 等文件。

#### 🛠️ 安裝必要套件
- `langchain`、`langchain-community`：語言模型與資料鏈整合框架
- `pypdf`、`python-docx`：處理 PDF 與 Word 檔案
- `sentence-transformers`：用於文字向量嵌入生成
- `faiss-cpu`：建立與查詢高效向量資料庫


In [ ]:
import os
upload_dir = "uploaded_docs"
os.makedirs(upload_dir, exist_ok=True)

In [ ]:
!pip install -U langchain langchain-community pypdf python-docx sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader, UnstructuredWordDocumentLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

#### **2.2 文件載入與向量模型定義**

此區段包含：
1. 自定義的 E5 嵌入模型類別 `CustomE5Embedding`
2. 讀取上傳之 `.txt`、`.pdf`、`.docx` 文件

#### 🧠 自定義 E5 向量模型
- 使用 `HuggingFaceEmbeddings` 建立子類別。
- 為每段文件自動加上 `passage:` 前綴，提升 E5 模型語意辨識效果。
- 為查詢句加上 `query:` 前綴，符合模型訓練邏輯。

#### 📄 載入多格式文件
- 自動偵測並載入三種檔案格式：
  - `.txt` → `TextLoader`
  - `.pdf` → `PyPDFLoader`
  - `.docx` → `UnstructuredWordDocumentLoader`
- 所有內容統一轉為 `documents`，用於後續分割與向量化。


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts] #當文字要轉為向量時，自動加入前綴詞 passage
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}") #當問題要轉為向量時，自動加入前綴詞 passage

In [ ]:
folder_path = upload_dir
documents = []
for file in os.listdir(folder_path):
    path = os.path.join(folder_path, file)
    if file.endswith(".txt"):
        loader = TextLoader(path)
    elif file.endswith(".pdf"):
        loader = PyPDFLoader(path)
    elif file.endswith(".docx"):
        loader = UnstructuredWordDocumentLoader(path)
    else:
        continue
    documents.extend(loader.load())

#### **2.3 文字分割與向量化處理**

本段程式負責將已載入的文件切分為小段，並轉換為向量，建立向量資料庫。

#### ✂️ 分割文字段落
- 使用 `RecursiveCharacterTextSplitter` 進行內容切分。
- `chunk_size=500`：每段最多 500 字元。
- `chunk_overlap=50`：每段與前一段重疊 50 字元，有助於上下文連貫。

#### 🧠 建立向量資料庫
- 使用前述自定義的 `CustomE5Embedding` 模型（支援中英文）。
- 將分段文字轉換為向量，並以 `FAISS` 建立向量資料庫 `vectorstore`。


In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50) # 500字切一段 100個字疊（可以自己調整）
split_docs = splitter.split_documents(documents)

In [ ]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")
vectorstore = FAISS.from_documents(split_docs, embedding_model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/498k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

#### **2.4 儲存與壓縮向量資料庫**

完成向量化後，將 `FAISS` 向量資料庫儲存至本地端並打包壓縮，方便日後載入與部署使用。

#### 🗂 儲存向量資料庫
- 使用 `save_local("faiss_db")` 將資料庫存至指定資料夾。

#### 📦 建立壓縮檔
- 使用 `zip` 指令將 `faiss_db` 資料夾打包成 `faiss_db_final.zip`。
- 可作為後續部署、上傳或版本管理使用。


In [ ]:
vectorstore.save_local("faiss_db")

In [ ]:
!zip -r faiss_db_final.zip faiss_db

  adding: faiss_db/ (stored 0%)
  adding: faiss_db/index.faiss (deflated 8%)
  adding: faiss_db/index.pkl (deflated 80%)
